In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'Iris.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
iris = pd.read_csv(file_content_stream)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
# changing labels of numbers 
iris['species_numb'] = np.where(iris['Species'] == 'Iris-virginica', 1, np.where(iris['Species'] == 'Iris-versicolor', 2, 3))

In [7]:
# defining inputs and target 
x = iris.drop(columns = ['Id', 'Species', 'species_numb'])
y = iris['species_numb']

# splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [8]:
#scaling the data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [14]:
# building random forest model 
one_vs_one_logit = OneVsOneClassifier(estimator = LogisticRegression()).fit(x_train, y_train)
#predicting on the test
one_vs_one_logit_pred = one_vs_one_logit.predict(x_test)

print(confusion_matrix(y_test, one_vs_one_logit_pred))
print(classification_report(y_test, one_vs_one_logit_pred))

[[ 9  1  0]
 [ 0 10  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



In [16]:
# building random forest model 
one_vs_one_tree = OneVsOneClassifier(estimator = DecisionTreeClassifier(max_depth = 3)).fit(x_train, y_train)
#predicting on the test
one_vs_one_tree_pred = one_vs_one_tree.predict(x_test)

print(confusion_matrix(y_test, one_vs_one_tree_pred))
print(classification_report(y_test, one_vs_one_tree_pred))

[[ 9  1  0]
 [ 1  9  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10
           3       1.00      1.00      1.00        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



In [ ]:
# based on these results I would choose logistic regression because it had better performace metrics